# OCR with PyTorch

### Importing Libraries

In [47]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

In [49]:
import warnings
warnings.filterwarnings('ignore')

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

### Loading Data

In [6]:
data = torchvision.datasets.ImageFolder(
    root = "./English/Fnt",
    transform = transforms.Compose([
        transforms.Resize((48, 48)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5,))
    ])
)

In [9]:
"""
Data Pipeline to Prepare Train, Test and Validation Data
"""

def load_split(dataset, batch_size, test_split=0.3):
    
    shuffle_dataset = True
    random_seed = 42
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(test_split * dataset_size))
    
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    train_indices, test_indices = indices[split:], indices[:split]
    
    testset_size = len(test_indices)
    indices = list(range(testset_size))
    split = int(np.floor(0.5 * testset_size))
    
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    val_indices, test_indices = indices[split:], indices[:split]
    
    # Create Data Smplers and Loaders    
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    test_sampler = torch.utils.data.SubsetRandomSampler(test_indices)
    
    train_loader = torch.utils.data.DataLoader(dataset, batch_size, sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(dataset, batch_size, sampler=test_sampler)
    val_loader = torch.utils.data.DataLoader(dataset, batch_size, sampler=val_sampler)
    
    return train_loader, test_loader, val_loader

In [10]:
batch_size = 36

train_loader, test_loader, val_loader = load_split(data, batch_size, test_split=0.3)

### Building Model

In [32]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64*9*9, 62)
        
        self.max_pool = nn.MaxPool2d(2, 2,ceil_mode=True)
        self.dropout = nn.Dropout(0.2)
        
        self.conv_bn1 = nn.BatchNorm2d(48, 3)
        self.conv_bn2 = nn.BatchNorm2d(16)
        self.conv_bn3 = nn.BatchNorm2d(32)
        self.conv_bn4 = nn.BatchNorm2d(64)
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        x = self.conv_bn2(x)
        
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)
        x = self.conv_bn3(x)
        
        x = F.relu(self.conv3(x))
#         x = self.max_pool(x)
        x = self.conv_bn4(x)
    
        x = x.view(-1,64 * 9 * 9)
        
        x = self.dropout(x)
        x = self.fc1(x)
        return x

In [33]:
def one_hot(lables, pred_size):
    one_hot_encoded = torch.zeros(len(labels), pred_size)
    
    y = 0
    for x in lables:
        one_hot_encoded[y][x] = 1
        y += 1
    return one_hot_encoded

In [34]:
class MyLoss(torch.autograd.Function):
    @staticmethod
    def forward(ctx, pred, lables):
        y = one_hot(lables, len(pred[0]))
        if torch.cuda.is_available():
            y = y.cuda()
        ctx.save_for_backward(y, pred)
        loss = - y * torch.log(pred)
        loss = loss.sum()/len(lables)
        return loss
    
    @staticmethod
    def backward(ctx, grad_output):
        y, pred = ctx.saved_tensors
        grad_input = (-y / pred) - y
        grad_input = grad_input/len(pred)
        return grad_input, grad_output

In [41]:
class MyCEL(nn.Module):
    
    def __init__(self):
        super(MyCEL, self).__init__()
        
    def forward(self, pred, labels):
        y = one_hot(labels, len(pred[0]))
        if torch.cuda.is_available():
            y = y.cuda()
        loss = - y * torch.log(pred)
        loss = loss.sum()/len(labels)
        return loss

In [42]:
network = Network()
use_cuda = True
if use_cuda and torch.cuda.is_available():
    network.cuda()
    print("cuda utilized")

In [51]:
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9)

epoch = 0
max_epoch = 5
end = False
myloss = MyCEL()
while epoch < max_epoch and not end:
    epoch += 1
    total_loss = 0
    total_correct = 0
    total_val = 0
    total_train = 0
    
    for data in tqdm(train_loader):
        
        images, labels = data
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels= labels.cuda()
            
        pred = network(images)
        pred = F.softmax(pred)
        
        loss = myloss(pred, labels)
        total_loss += loss.item()
        total_train += len(pred)
#         Reset optimizer grads
        optimizer.zero_grad()
#     Calculate gradients
        loss.backward()
#     Gradient descent
        optimizer.step()
    
        total_correct += pred.argmax(dim = 1).eq(labels).sum()
        
    print("epoch :", epoch, "Training Accuracy : ", total_correct*1.0/total_train, "Train Loss", total_loss*1.0/len(train_loader))
    
    if total_correct*1.0/total_train >= 0.98:
        end = True
        
    total_loss = 0
    val_total_correct = 0
    
    for batch in tqdm(val_loader):
        
        images, labels = batch
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels= labels.cuda()  
        
        pred = network(images)
        loss = F.cross_entropy(pred, labels)
        total_loss+= loss.item()
        total_val += len(pred)
        val_total_correct += pred.argmax(dim=1).eq(labels).sum()
        print("epoch : ", epoch, "Val Accuracy :", val_total_correct*1.0/total_val, "Val Loss : ", total_loss*1.0/len(val_loader))
    torch.cuda.empty_cache()

100%|██████████████████████████████████████████████████████████████████████████████| 1225/1225 [01:45<00:00, 11.62it/s]


epoch : 1 Training Accuracy :  

  0%|                                                                                          | 0/263 [00:00<?, ?it/s]

tensor(0.8459) Train Loss 0.49068624040301967
epoch :  1 Val Accuracy : 

  1%|▉                                                                                 | 3/263 [00:00<00:28,  9.12it/s]

...

epoch :  5 Val Accuracy : tensor(0.9389) Val Loss :  0.19573371068436382
epoch :  5 Val Accuracy : tensor(0.9388) Val Loss :  0.19664337246646899
epoch :  5 Val Accuracy : tensor(0.9388) Val Loss :  0.19714084569322293
epoch :  5 Val Accuracy : 

 99%|███████████████████████████████████████████████████████████████████████████████ | 260/263 [00:18<00:00, 14.98it/s]

tensor(0.9391) Val Loss :  0.19737120433055402
epoch :  5 Val Accuracy : tensor(0.9389) Val Loss :  0.19915384690174584
epoch :  5 Val Accuracy : tensor(0.9387) Val Loss :  0.200327893527855
epoch :  5 Val Accuracy : 

100%|████████████████████████████████████████████████████████████████████████████████| 263/263 [00:18<00:00, 14.23it/s]

tensor(0.9389) Val Loss :  0.2005291217690185
epoch :  5 Val Accuracy : tensor(0.9388) Val Loss :  0.20170185924596207
epoch :  5 Val Accuracy : tensor(0.9387) Val Loss :  0.20248533867497862


In [52]:
test_total_correct = 0
total_test = 0
x = 0

for batch in tqdm(test_loader):
    images, labels = batch
    
    if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels= labels.cuda()
            
    pred = network(images)
    total_test += len(pred)
    x += 1
    test_total_correct+= pred.argmax(dim = 1).eq(labels).sum()

print("Test Accuracy : ", test_total_correct*1.0/total_test)

100%|████████████████████████████████████████████████████████████████████████████████| 263/263 [00:27<00:00,  9.51it/s]

Test Accuracy :  tensor(0.9381)


In [53]:
PATH = "entire_model.pt"

torch.save(network, PATH)